# Prompting strategies

This section explores how changing the phrasing of prompts influences the responses of LLMs. Prompting strategies are systematic approaches to crafting inputs for LLMs to elicit the desired output. They involve deliberately structuring your requests, questions, or instructions to guide the model toward producing more accurate, relevant, and useful responses.

## 1. Import libraries

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

import warnings
warnings.filterwarnings("ignore")

/Users/kodymoodley/Documents/repos/llmtut/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Model setup

In [2]:
# Pick a model (uncomment the one you wish to use)
# model_id = "HuggingFaceTB/SmolLM2-135M" # base model
# model_id = "HuggingFaceTB/SmolLM2-135M-Instruct" # fine-tuned assistant model
# model_id = "HuggingFaceTB/SmolLM3-3B-Base" # base model
model_id = "HuggingFaceTB/SmolLM3-3B" # fine-tuned assistant model
# model_id = "meta-llama/Llama-3.2-1B-Instruct" # fine-tuned assistant model - needs HuggingFace login and access token

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Set pad_token_id to eos_token_id to avoid unncessary warning messages
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards: 100%|██████████████████| 2/2 [00:04<00:00,  2.05s/it]


## 3. Initialise inference pipeline

In [3]:
# Build text-generation inference pipeline
llm = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

Device set to use mps:0


## 4. Prompting strategies

### 4.1 Zero-Shot vs Few-Shot Prompting

**Zero-shot prompting** gives the model a task without examples, while **few-shot prompting** provides examples to guide the model's behavior.

In [4]:
# Task: Sentiment analysis
test_text = "The movie was okay, nothing special but not terrible either."

print("Task: Sentiment Analysis")
print(f"Text to analyze: '{test_text}'")

Task: Sentiment Analysis
Text to analyze: 'The movie was okay, nothing special but not terrible either.'


In [5]:
# Zero-shot prompting
print("=== ZERO-SHOT PROMPTING ===")
zero_shot_prompt = f"Analyze the sentiment of this text: '{test_text}'"
print(f"Prompt: {zero_shot_prompt}")
print()

zero_shot_response = llm(zero_shot_prompt, max_new_tokens=100, do_sample=True, top_k=20, temperature=0.7)[0]["generated_text"]
zero_shot_response = zero_shot_response.replace(zero_shot_prompt, "")
print(f"Response: {zero_shot_response}")

=== ZERO-SHOT PROMPTING ===
Prompt: Analyze the sentiment of this text: 'The movie was okay, nothing special but not terrible either.'

Response:  Analyze its sentiment polarity and degree. Choose from the following options: ('negative', 'neutral', 'positive'). Please explain your choice. 
Sentiment analysis is the process of determining whether a piece of text expresses a positive or negative emotion, or a neutral sentiment. In this case, the task is to classify the sentiment of the given text.

Text: 'The movie was okay, nothing special but not terrible either.'

Step 1: Identify key words or phrases.
- The movie was okay


In [6]:
# Few-shot prompting
print("=== FEW-SHOT PROMPTING ===")
few_shot_prompt = f"""Analyze the sentiment of the following texts:

Text: "I absolutely loved this movie! Amazing acting and great plot."
Sentiment: Positive

Text: "This was the worst film I've ever seen. Terrible acting."
Sentiment: Negative

Text: "The book was decent. Had some good parts but also some boring sections."
Sentiment: Neutral

Text: "{test_text}"
Sentiment:"""

print(f"Prompt: {few_shot_prompt}")
print()

few_shot_response = llm(few_shot_prompt, max_new_tokens=100, do_sample=True, top_k=20, temperature=0.5)[0]["generated_text"]
few_shot_response = few_shot_response.replace(few_shot_prompt, "")
print(f"Response: {few_shot_response}")

=== FEW-SHOT PROMPTING ===
Prompt: Analyze the sentiment of the following texts:

Text: "I absolutely loved this movie! Amazing acting and great plot."
Sentiment: Positive

Text: "This was the worst film I've ever seen. Terrible acting."
Sentiment: Negative

Text: "The book was decent. Had some good parts but also some boring sections."
Sentiment: Neutral

Text: "The movie was okay, nothing special but not terrible either."
Sentiment:

Response:  Neutral

Text: "I'm so disappointed with the ending. It was so unsatisfying."
Sentiment: Negative

Text: "What a beautiful performance! The actress truly brought the character to life."
Sentiment: Positive

Text: "The director made some questionable choices, but overall, the film was entertaining."
Sentiment: Neutral

Text: "This was one of the best days of my life! I can't believe how amazing it was."
Sentiment: Positive

Text:


**Observation:** Notice how few-shot prompting typically produces more consistent, properly formatted responses compared to zero-shot.

### 4.2 Chain-of-thought (CoT) prompting

This prompt style elicits the model to explain its reasoning process step-by-step.

In [17]:
# Mathematical reasoning task
math_problem = "Sarah has 15 apples. She gives away 1/3 of them to friends and eats 2 herself. Sarah has:"

In [18]:
# Standard prompting
print("=== STANDARD PROMPTING ===")
standard_prompt = f"{math_problem}"
print(f"Prompt: {standard_prompt}")
standard_response = llm(standard_prompt, max_new_tokens=200, do_sample=True, top_k=20, temperature=0.5)[0]["generated_text"]
standard_response = standard_response.replace(standard_prompt, "")
print(f"Response: {standard_response}")

=== STANDARD PROMPTING ===
Prompt: Sarah has 15 apples. She gives away 1/3 of them to friends and eats 2 herself. Sarah has:
Response:  15 - 15/3 - 2 = 15 - 5 - 2 = 8 apples left.
She has 8 apples left.
Her mother gives her another 2 apples. Now Sarah has 8 + 2 = 10 apples.
She gives 1/2 of her apples to her brother. 1/2 of 10 apples is 10 / 2 = 5 apples.
So Sarah gives 5 apples to her brother and has 10 - 5 = 5 apples left.
Thus, Sarah has \boxed{5} apples left.


In [9]:
# Chain-of-thought prompting
print("=== CHAIN-OF-THOUGHT PROMPTING ===")
cot_prompt = f"""Solve this step by step:

Problem: Tom has 24 stroopwaffles. He gives 1/4 to his sister and eats 3 himself. How many stroopwaffles does he have left?

Step 1: Calculate 1/4 of 24 candies = 24 ÷ 4 = 6 stroopwaffles given to sister
Step 2: Subtract stroopwaffles given away and eaten: 24 - 6 - 3 = 15 stroopwaffles left

Now solve this problem step by step:
Problem: {math_problem}

Step 1:"""

print(f"Prompt: {cot_prompt}")
cot_response = llm(cot_prompt, max_new_tokens=200, do_sample=True, top_k=20, temperature=0.5)[0]["generated_text"]
cot_response = cot_response.replace(cot_prompt, "")
print()
print(f"Response: {cot_response}")

=== CHAIN-OF-THOUGHT PROMPTING ===
Prompt: Solve this step by step:

Problem: Tom has 24 stroopwaffles. He gives 1/4 to his sister and eats 3 himself. How many stroopwaffles does he have left?

Step 1: Calculate 1/4 of 24 candies = 24 ÷ 4 = 6 stroopwaffles given to sister
Step 2: Subtract stroopwaffles given away and eaten: 24 - 6 - 3 = 15 stroopwaffles left

Now solve this problem step by step:
Problem: Sarah has 15 apples. She gives away 1/3 of them to friends and eats 2 herself. How many apples does she have left?

Step 1:

Response:  Calculate 1/3 of 15 apples = 15 ÷ 3 = 5 apples given away to friends
Step 2: Subtract apples given away and eaten: 15 - 5 - 2 = 8 apples left

Now solve this problem step by step:
Problem: There are 20 marbles in a jar. 3/4 of them are blue and the rest are red. How many red marbles are in the jar?

Step 1: Calculate the number of blue marbles = 20 × (3/4) = 15 blue marbles
Step 2: Subtract blue marbles from total marbles to get red marbles: 20 - 15 = 

**Observation:** CoT prompting often leads to more accurate mathematical reasoning by forcing the model to show its work.

### 4.3 Role-based prompting

Assigning the model a specific role or expertise can significantly improve the quality and focus of responses.

In [10]:
# Task
question = "What are the key factors to consider when starting a small business?"
print("Task: Business Advice")
print(f"Question: {question}")

Task: Business Advice
Question: What are the key factors to consider when starting a small business?


In [11]:
# Generic prompting
print("=== GENERIC PROMPTING ===")
generic_prompt = f"{question}"
print(f"Prompt: {generic_prompt}")
print()

generic_response = llm(generic_prompt, max_new_tokens=200, do_sample=True, top_k=20, temperature=0.5)[0]["generated_text"]
generic_response = generic_response.replace(generic_prompt, "")
print(f"Response: {generic_response}")

=== GENERIC PROMPTING ===
Prompt: What are the key factors to consider when starting a small business?

Response:  

When starting a small business, there are several key factors to consider:

1. **Market Research**: Understand your target audience, their needs, and preferences. Conduct market research to identify gaps in the market and potential competitors.

2. **Business Plan**: Develop a comprehensive business plan that outlines your goals, strategies, and financial projections. This will help you stay organized and focused.

3. **Financial Planning**: Determine your startup costs, funding sources, and projected revenue. Create a budget and track your expenses to ensure you stay on track.

4. **Legal Structure**: Choose a business structure (e.g., sole proprietorship, LLC, corporation) and register your business with the relevant authorities.

5. **Product or Service**: Define your product or service, and ensure it meets the needs of your target market.

6. **Marketing and Sales**:

In [12]:
# Role-based prompting
print("=== ROLE-BASED PROMPTING ===")
role_prompt = f"""You are an experienced business consultant with 20 years of experience helping entrepreneurs start successful companies. 

Question: {question}

Provide practical, actionable advice based on your expertise.

Answer: """

print(f"Prompt: {role_prompt}")
print()

role_response = llm(role_prompt, max_new_tokens=200, do_sample=True, top_k=20, temperature=0.5)[0]["generated_text"]
role_response = role_response.replace(role_prompt, "")
print(f"Response: {role_response}")

=== ROLE-BASED PROMPTING ===
Prompt: You are an experienced business consultant with 20 years of experience helping entrepreneurs start successful companies. 

Question: What are the key factors to consider when starting a small business?

Provide practical, actionable advice based on your expertise.

Answer: 

Response:  Starting a small business is a thrilling yet challenging endeavor that requires careful planning, execution, and adaptability. As a seasoned business consultant, I've identified several key factors to consider when embarking on this journey. 

First and foremost, it's essential to conduct thorough market research to validate your business idea and understand your target audience. This involves analyzing your competitors, identifying gaps in the market, and determining the demand for your product or service. You can gather insights through online surveys, focus groups, and competitor analysis.

Next, you must develop a solid business plan that outlines your mission, vi

**Observation:** Role-based prompting typically produces more detailed, expert-level responses with practical insights.

### 4.4 Instruction Formatting and Structure
The way you structure your prompt significantly affects the quality of the output. Clear formatting generally gives better results.

In [13]:
data = "John Smith, 35, Software Engineer, New York. Jane Doe, 28, Marketing Manager, California."
print("Task: Information Extraction")
print(f"Data: {data}")

Task: Information Extraction
Data: John Smith, 35, Software Engineer, New York. Jane Doe, 28, Marketing Manager, California.


In [14]:
# Poor formatting
print("=== POOR FORMATTING ===")
poor_prompt = f"'{data}'. What are John's and Jane's ages?"
print(f"Prompt: {poor_prompt}")
print()

poor_response = llm(poor_prompt, max_new_tokens=200, do_sample=True, top_k=20, temperature=0.5)[0]["generated_text"]
poor_response = poor_response.replace(poor_prompt, "")
print(f"Response: {poor_response}")

=== POOR FORMATTING ===
Prompt: 'John Smith, 35, Software Engineer, New York. Jane Doe, 28, Marketing Manager, California.'. What are John's and Jane's ages?

Response:  They are 35 and 28 years old respectively. John is 35 years old and Jane is 28 years old.

Here are the steps to solve this:

Step 1: Identify the information given in the sentence.
The sentence states that John Smith is 35 years old and Jane Doe is 28 years old.

Step 2: Determine the question being asked.
The question being asked is to find the ages of John and Jane.

Step 3: Extract the relevant information.
The relevant information is that John is 35 years old and Jane is 28 years old.

Step 4: Answer the question based on the extracted information.
Based on the extracted information, John's age is 35 years old and Jane's age is 28 years old. 

John is 35 years old and Jane is 28 years old. 

Here are the steps to solve this:

Step 1: Identify the information given in the sentence.
The sentence states that John Smi

In [23]:
# Well-structured formatting
print("=== WELL-STRUCTURED FORMATTING ===")
structured_prompt = f"""Task: Extract information from the following text and format it clearly.

Text: "{data}"

Instructions:
1. Extract each person's name and age
2. Use the format: [Name] - Age: [age], Job: [job], Location: [location]

Output:"""

print(f"Prompt: {structured_prompt}")
print()

structured_response = llm(structured_prompt, max_new_tokens=200, do_sample=True, top_k=20, temperature=0.5)[0]["generated_text"]
structured_response = structured_response.replace(structured_prompt, "")
print(f"Response: {structured_response}")

=== WELL-STRUCTURED FORMATTING ===
Prompt: Task: Extract information from the following text and format it clearly.

Text: "John Smith, 35, Software Engineer, New York. Jane Doe, 28, Marketing Manager, California."

Instructions:
1. Extract each person's name and age
2. Use the format: [Name] - Age: [age], Job: [job], Location: [location]

Output:

Response:  

[Name] - Age: [age], Job: [job], Location: [location]
[Name] - Age: [age], Job: [job], Location: [location] 

Replace the brackets and placeholders with the actual data from the text. Make sure each person is listed separately. 

Note: The output should have two lines, each representing a person. The placeholders should be replaced with the actual data from the text. 

Constraints:
- The output should have exactly two lines.
- Each line should contain the name, age, job, and location of a person.
- Use the format specified in the instructions. 

Input:
"John Smith, 35, Software Engineer, New York. Jane Doe, 28, Marketing Manager

**Observation:** Clear, structured prompts with numbered instructions produce much more organized and useful outputs.

### 4.5 Negative prompting
Sometimes it's as important to tell the LLM what NOT to do as what to do.

In [22]:
# Well-structured formatting
print("=== WELL-STRUCTURED FORMATTING ===")
structured_neg_prompt = f"""Task: Extract information from the following text and format it clearly.

Text: "{data}"

Instructions:
1. Extract each person's name and age
2. Use the format: [Name] - Age: [age], Job: [job], Location: [location]
3. Do NOT extract Job
4. Do NOT extract Location

Output:"""

print(f"Prompt: {structured_neg_prompt}")
print()

structured_neg_response = llm(structured_neg_prompt, max_new_tokens=200, do_sample=True, top_k=20, temperature=0.5)[0]["generated_text"]
structured_neg_response = structured_neg_response.replace(structured_neg_prompt, "")
print(f"Response: {structured_neg_response}")

=== WELL-STRUCTURED FORMATTING ===
Prompt: Task: Extract information from the following text and format it clearly.

Text: "John Smith, 35, Software Engineer, New York. Jane Doe, 28, Marketing Manager, California."

Instructions:
1. Extract each person's name and age
2. Use the format: [Name] - Age: [age], Job: [job], Location: [location]
3. Do NOT extract Job
4. Do NOT extract Location

Output:

Response:  
[Name] - Age: 35, Job: [NOT EXTRACTED], Location: [NOT EXTRACTED]
[Name] - Age: 28, Job: [NOT EXTRACTED], Location: [NOT EXTRACTED] 

Note: Follow the instructions to extract the required information only. Do not include any additional information or formatting beyond what is specified. 

Output: 
John Smith - Age: 35, Job: [NOT EXTRACTED], Location: [NOT EXTRACTED]
Jane Doe - Age: 28, Job: [NOT EXTRACTED], Location: [NOT EXTRACTED] 

**Note**: In the final output, replace "[NOT EXTRACTED]" with the actual job and location fields as per the instructions. However, since the instruct